## Parameter Variable Settings

This Jupyter Notebook processes data from RGB, RAW, and event camera sensor.

Functions:
1. Loads RGB, RAW, and DVS frames and event streams.
2. Aligns timestamps across different sensor modalities for accurate comparison.
4. Detects target regions (i.e., AprilTag, Barbara tag).
5. Crops and filters event data based on detected regions.
6. Visualizes frames, event data, cropping results, and target trajectories.
7. Analyzes event statistics, including per-pixel time intervals and frequency spectrum.
8. Playback for filtered event data.


In [ ]:
import dv_processing as dv
import torch
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm
import os
from matplotlib import pyplot as plt
import mmap
import aiofiles
import sys
from datetime import datetime
sys.path.append('./src/process_data')  # Add module path
import process_data.file_read as file_read
import process_data.dvs_generate as dvs_generate
import process_data.tag_detector as tag_detector
import process_data.event_filter as event_filter
import cv2
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import math

INPUT_FOLDER = ""  # Replace with your folder path
FILE_SUFFIX = "170406"  # Replace with your file suffix

# Find matching files
try:
    files = file_read.find_matching_files(INPUT_FOLDER, FILE_SUFFIX)        
except FileNotFoundError as e:
    print(f"Error: {str(e)}")

# Width and height used when reading pi data
PI_IMAGE_WIDTH = 692
PI_IMAGE_HEIGHT = 520

# K values used for generating simulated event data
# Used for generating raw simulated event data (default is dvs original k)
k_values_raw = [2.388, 4.166e-7, 1.541e-6, 9.768e-8, 1.466e-11, 9.824e-6]
# Used for generating rgb simulated event data (default is dvs original k)
k_values_rgb = [5.332474147628972,0.9003332027266823,8.288263352543993e-06,1.0992397172828087e-07,3.302652963977818e-09,1.1012444038716504e-07] 

# Ratio for tag detection trajectory
TAG_REF_WIDTH = 287      # AprilTag reference width (pixels)
BARBARA_REF_SIZE = 861   # Barbara reference side length (pixels)
BARBARA_GAP = 82         # Gap between Barbara and tag (pixels)
margin_ratio = 0.03

# Number of threads
N_WORKERS = 4 

# Batch size for filtering event data
BATCH_SIZE_FOR_EVENT = 100000

## Main Process

In [ ]:
############################################################
#                      Data Loading Section                #
############################################################
dv_frames, dv_frames_timestamps = file_read.load_frames(files['dv'])
dv_events_tensor = file_read.load_events(files['dv'])
rgb_frames = file_read.read_rgb_frames(files['rgb_frames'], PI_IMAGE_HEIGHT, PI_IMAGE_WIDTH)
raw_frames = file_read.read_raw_frames(files['raw_frames'], PI_IMAGE_HEIGHT, PI_IMAGE_WIDTH)
pi_timestamps, real_timestamps = file_read.read_metadata(files['metadata'])

############################################################
#                   Generate Simulated Event Data          #
############################################################
# Single-threaded RGB event generation
rgb_events_tensor = dvs_generate.generate_events_tensor(
pi_timestamps,  # PI camera timestamps
rgb_frames,     # RGB frame data
is_rgb=True,    # Mark as RGB data
k_values=k_values_rgb
)
# Single-threaded RAW event generation
raw_events_tensor = dvs_generate.generate_events_tensor(
pi_timestamps,  # PI camera timestamps
raw_frames,     # RAW frame data
is_rgb=False,   # Mark as non-RGB (i.e., RAW) data
k_values=k_values_raw
)

############################################################
#                      Time Alignment                      #
############################################################
# Calculate time offset
time_offset = file_read.calculate_time_offset(pi_timestamps, real_timestamps)
# Adjust DV frame timestamps
dv_frames_timestamps = dv_frames_timestamps - time_offset
# Adjust DV event timestamps
dv_events_tensor[:, 0] = dv_events_tensor[:, 0] - time_offset


############################################################
#                   Trajectory Data (Parallel)             #
############################################################
# Construct parameter lists
margin_ratios = [margin_ratio] * N_WORKERS
tag_ref_widths = [TAG_REF_WIDTH] * N_WORKERS
barbara_ref_sizes = [BARBARA_REF_SIZE] * N_WORKERS
barbara_gaps = [BARBARA_GAP] * N_WORKERS
is_raws_rgb = [False] * N_WORKERS
is_raws_raw = [True] * N_WORKERS

rgb_frame_batches = tag_detector.split_batches(rgb_frames.numpy(), N_WORKERS)
raw_frame_batches = tag_detector.split_batches(raw_frames.numpy(), N_WORKERS)
ts_batches = tag_detector.split_batches(pi_timestamps, N_WORKERS)

dv_frame_batches = tag_detector.split_batches(dv_frames.numpy(), N_WORKERS)
dv_ts_batches = tag_detector.split_batches(dv_frames_timestamps, N_WORKERS)

# Batch get RGB frame crop box information
# Multi-threaded batch processing with progress display
with ThreadPoolExecutor(max_workers=N_WORKERS) as executor:
    rgb_all_results = []
    for batch_result in tqdm(
        executor.map(
            tag_detector.process_batch,
            rgb_frame_batches, ts_batches,
            margin_ratios, tag_ref_widths, barbara_ref_sizes, barbara_gaps, is_raws_rgb
        ),
        total=N_WORKERS
    ):
        rgb_all_results.append(batch_result)

rgb_crops_info = [item for batch in rgb_all_results for item in batch]# Merge all results
rgb_crops_info.sort(key=lambda x: x[2])  # Sort by timestamp


# Batch get RAW frame crop box information
# Multi-threaded batch processing with progress display
with ThreadPoolExecutor(max_workers=N_WORKERS) as executor:
    raw_all_results = []
    for batch_result in tqdm(
        executor.map(
            tag_detector.process_batch,
            raw_frame_batches, ts_batches,
            margin_ratios, tag_ref_widths, barbara_ref_sizes, barbara_gaps, is_raws_raw
        ),
        total=N_WORKERS
    ):
        raw_all_results.append(batch_result)

raw_crops_info = [item for batch in raw_all_results for item in batch]# Merge all results
raw_crops_info.sort(key=lambda x: x[2])  # Sort by timestamp

# Batch get DV frame crop box information
# Multi-threaded batch processing with progress display
with ThreadPoolExecutor(max_workers=N_WORKERS) as executor:
    dv_all_results = []
    for batch_result in tqdm(
        executor.map(
            tag_detector.process_batch,
            dv_frame_batches, dv_ts_batches,
            margin_ratios, tag_ref_widths, barbara_ref_sizes, barbara_gaps, is_raws_rgb
        ),
        total=N_WORKERS
    ):
        dv_all_results.append(batch_result)

dv_crops_info = [item for batch in dv_all_results for item in batch]# Merge all results
dv_crops_info.sort(key=lambda x: x[2])  # x[2] is timestamp# Sort by timestamp
# Column 0: barbara_info - Contains detected Barbara tag information, such as polygon vertices, rotation angle, center point, etc.
# Column 1: tag_info - Contains detected AprilTag information, such as ID, position, etc.
# Column 2: timestamp - Current frame timestamp


############################################################
#                   Filter Event Data                      #
############################################################
# Assume box_size is a tuple (w, h)
def round_up_to_10(x):
    return int(math.ceil(x / 10.0) * 10)

# Round up to the nearest multiple of 10 as the maximum side length for filtering event data
RGB_BOX_SIZE_FOR_EVENT = round_up_to_10(max(rgb_crops_info[0][0]['polygon'].ptp(axis=0)))
RAW_BOX_SIZE_FOR_EVENT = round_up_to_10(max(raw_crops_info[0][0]['polygon'].ptp(axis=0)))
DV_BOX_SIZE_FOR_EVENT = round_up_to_10(max(dv_crops_info[0][0]['polygon'].ptp(axis=0)))

# RGB event filtering
# Use parallel processing function
filtered_events_rgb = event_filter.filter_events_parallel(
    events_tensor=rgb_events_tensor,  # Your event data
    crops_info=rgb_crops_info,        # Crop box information
    target_size=RGB_BOX_SIZE_FOR_EVENT,  # Target crop box size
    transform=True,                    # Whether to transform coordinates
    batch_size=BATCH_SIZE_FOR_EVENT,                # Number of events processed per batch
    n_workers=N_WORKERS                       # Number of parallel processes
)

# RAW event filtering
# Use parallel processing function
filtered_events_raw = event_filter.filter_events_parallel(
    events_tensor=raw_events_tensor,  # Your event data
    crops_info=raw_crops_info,        # Crop box information
    target_size=RAW_BOX_SIZE_FOR_EVENT,  # Target crop box size
    transform=True,                    # Whether to transform coordinates
    batch_size=BATCH_SIZE_FOR_EVENT,                # Number of events processed per batch
    n_workers=N_WORKERS                       # Number of parallel processes
)

# DV event filtering
# Use parallel processing function
filtered_events_dv = event_filter.filter_events_parallel(
    events_tensor=dv_events_tensor,  # Your event data
    crops_info=dv_crops_info,        # Crop box information
    target_size=DV_BOX_SIZE_FOR_EVENT,  # Target crop box size
    transform=True,                    # Whether to transform coordinates
    batch_size=BATCH_SIZE_FOR_EVENT,                # Number of events processed per batch
    n_workers=N_WORKERS                       # Number of parallel processes
)


############################################################
#                      End                                 #
############################################################

## Loading Visualization

In [ ]:
# Create a figure with two subplots
plt.figure(figsize=(15, 6))  # Adjust overall figure size

# First subplot: RAW frame
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.imshow(raw_frames[0].numpy(), cmap='gray')
plt.title('RAW Frame 0')
plt.axis('off')  # Turn off axes

# Second subplot: RGB frame
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.imshow(rgb_frames[0].numpy())
plt.title('RGB Frame 0')
plt.axis('off')  # Turn off axes

# Adjust spacing between subplots
plt.tight_layout()

# Display the figure
plt.show()

## Time Interval Histogram

In [ ]:
import process_data.interval_fit as interval_fit
import importlib
importlib.reload(interval_fit)


num_pixels_rgb, dt_rgb, mu_rgb, sigma_rgb = interval_fit.analyze_per_pixel_event_intervals_combined(
    events=filtered_events_rgb,
    min_events_per_pixel=10,         # For example, each pixel needs at least 10 events
    max_dt_us_for_plot=100000,          # For example, only look at intervals below 200 microseconds when plotting
    plot_bins=100,
    type='RGB'
)
plt.show()  # Display the figure

num_pixels_raw, dt_raw, mu_raw, sigma_raw = interval_fit.analyze_per_pixel_event_intervals_combined(
    events=filtered_events_raw,
    min_events_per_pixel=10,         # For example, each pixel needs at least 10 events
    max_dt_us_for_plot=100000,          # For example, only look at intervals below 200 microseconds when plotting
    plot_bins=100,
    type='RAW'
)
plt.show()  # Display the figure


num_pixels_dv, dt_dv, mu_dv, sigma_dv = interval_fit.analyze_per_pixel_event_intervals_combined(
    events=filtered_events_dv,
    min_events_per_pixel=10,         # For example, each pixel needs at least 10 events
    max_dt_us_for_plot=100000,          # For example, only look at intervals below 200 microseconds when plotting
    plot_bins=100,
    type='DV'
)
plt.show()  # Display the figure

import process_data.interval_fit as interval_fit
import importlib
importlib.reload(interval_fit)

# Frequency spectrum (time intervals)
results = interval_fit.analyze_event_frequency_spectrum(
    filtered_events_raw,          # Event data
    max_freq_hz=100,              # Maximum frequency limit
    bins=50                       # Number of histogram bins
)
plt.show()


# FFT spectrum
fft_results = interval_fit.analyze_event_fft_spectrum(
    filtered_events_raw,
    sampling_rate=1000,   # Sampling rate (Hz)
    max_freq_hz=100       # Maximum frequency limit
)
plt.show()

## Crop Box Visualization

In [ ]:
# Create detector
detector = tag_detector.create_detector()

# Process RGB frame
barbara_info_rgb, cropped_rgb, ts_rgb = tag_detector.process_frame(
    rgb_frames[0].numpy(), pi_timestamps[0], detector, 
    margin_ratio=margin_ratio, 
    tag_ref_width=TAG_REF_WIDTH, 
    barbara_ref_size=BARBARA_REF_SIZE, 
    barbara_gap=BARBARA_GAP,
    is_raw=False
)

# Process RAW frame
barbara_info_raw, cropped_raw, ts_raw = tag_detector.process_frame(
    raw_frames[0].numpy(), pi_timestamps[0], detector, 
    margin_ratio=margin_ratio, 
    tag_ref_width=TAG_REF_WIDTH, 
    barbara_ref_size=BARBARA_REF_SIZE, 
    barbara_gap=BARBARA_GAP,
    is_raw=True
)

# Process DV frame
barbara_info_dv, cropped_dv, ts_dv = tag_detector.process_frame(
    dv_frames[0].numpy(), dv_frames_timestamps[0], detector, 
    margin_ratio=margin_ratio, 
    tag_ref_width=TAG_REF_WIDTH, 
    barbara_ref_size=BARBARA_REF_SIZE, 
    barbara_gap=BARBARA_GAP,
    is_raw=False
)

# Add the following three lines before displaying images
def get_box_size(polygon):
    if polygon is None:
        return None
    width = np.linalg.norm(polygon[1] - polygon[0])  # Bottom edge length
    height = np.linalg.norm(polygon[2] - polygon[1])  # Right edge length
    return (width, height)

print(f"RGB box size: {get_box_size(barbara_info_rgb['polygon'] if barbara_info_rgb else None)}")
print(f"RAW box size: {get_box_size(barbara_info_raw['polygon'] if barbara_info_raw else None)}")
print(f"DV box size: {get_box_size(barbara_info_dv['polygon'] if barbara_info_dv else None)}")


# Display single frame cropping effect
plt.figure(figsize=(12, 12))
# 1. RGB original image
plt.subplot(3, 2, 1)
frame_img_rgb = rgb_frames[0].numpy().copy()
plt.imshow(frame_img_rgb)
if barbara_info_rgb is not None:
    poly = barbara_info_rgb['polygon']
    x, y = poly[:,0], poly[:,1]
    plt.plot(x, y, '-', color='lime', linewidth=4)
    plt.plot([x[-1], x[0]], [y[-1], y[0]], '-', color='lime', linewidth=4)
plt.title('RGB Frame with Barbara Region')
plt.axis('off')

# 2. RGB cropped
plt.subplot(3, 2, 2)
if cropped_rgb is not None:
    plt.imshow(cropped_rgb)
    plt.title('RGB Cropped Frame')
else:
    plt.title('RGB Cropped Frame (None)')
plt.axis('off')

# 3. RAW original image
plt.subplot(3, 2, 3)
frame_img_raw = raw_frames[0].numpy().copy()
plt.imshow(frame_img_raw, cmap='gray')
if barbara_info_raw is not None:
    poly = barbara_info_raw['polygon']
    x, y = poly[:,0], poly[:,1]
    plt.plot(x, y, '-', color='lime', linewidth=4)
    plt.plot([x[-1], x[0]], [y[-1], y[0]], '-', color='lime', linewidth=4)
plt.title('RAW Frame with Barbara Region')
plt.axis('off')

# 4. RAW cropped
plt.subplot(3, 2, 4)
if cropped_raw is not None:
    plt.imshow(cropped_raw, cmap='gray')
    plt.title('RAW Cropped Frame')
else:
    plt.title('RAW Cropped Frame (None)')
plt.axis('off')

# 5. DV original image
plt.subplot(3, 2, 5)
frame_img_dv = dv_frames[0].numpy().copy()
plt.imshow(frame_img_dv, cmap='gray')
if barbara_info_dv is not None:
    poly = barbara_info_dv['polygon']
    x, y = poly[:,0], poly[:,1]
    plt.plot(x, y, '-', color='lime', linewidth=4)
    plt.plot([x[-1], x[0]], [y[-1], y[0]], '-', color='lime', linewidth=4)
plt.title('DV Frame with Barbara Region')
plt.axis('off')

# 6. DV cropped
plt.subplot(3, 2, 6)
if cropped_dv is not None:
    plt.imshow(cropped_dv, cmap='gray')
    plt.title('DV Cropped Frame')
else:
    plt.title('DV Cropped Frame (None)')
plt.axis('off')

plt.tight_layout()
plt.subplots_adjust(wspace=0.05, hspace=0.15)  # Reduce column spacing
plt.show()

## Trajectory Visualization

In [ ]:
plt.figure(figsize=(15, 10))

# Extract center point trajectories
rgb_centers = np.array([info[0]['center'] for info in rgb_crops_info if info[0] is not None])
raw_centers = np.array([info[0]['center'] for info in raw_crops_info if info[0] is not None])
dv_centers  = np.array([info[0]['center'] for info in dv_crops_info  if info[0] is not None])


# Extract angles
rgb_angles = np.array([info[0]['angle'] for info in rgb_crops_info if info[0] is not None])
raw_angles = np.array([info[0]['angle'] for info in raw_crops_info if info[0] is not None])
dv_angles  = np.array([info[0]['angle'] for info in dv_crops_info  if info[0] is not None])

rgb_times = [info[2] for info in rgb_crops_info if info[0] is not None]
raw_times = [info[2] for info in raw_crops_info if info[0] is not None]
dv_times  = [info[2] for info in dv_crops_info  if info[0] is not None]

plt.figure(figsize=(10, 8))

# 1. Combine trajectories into one plot
plt.subplot(2, 1, 1)
s =1 # Point size
lw = 1  # Line width

if len(rgb_centers) > 0:
    plt.plot(rgb_centers[:,0], rgb_centers[:,1], '-', color='red', lw=lw, label='RGB')
    plt.scatter(rgb_centers[:,0], rgb_centers[:,1], color='red', s=s)

if len(raw_centers) > 0:
    plt.plot(raw_centers[:,0], raw_centers[:,1], '-', color='green', lw=lw, label='RAW')
    plt.scatter(raw_centers[:,0], raw_centers[:,1], color='green', s=s)

if len(dv_centers) > 0:
    plt.plot(dv_centers[:,0], dv_centers[:,1], '-', color='blue', lw=lw, label='DV')
    plt.scatter(dv_centers[:,0], dv_centers[:,1], color='blue', s=s)

plt.title('Barbara Center Trajectories')
plt.xlabel('X')
plt.ylabel('Y')
plt.gca().invert_yaxis()
plt.axis('equal')
plt.grid(True)
plt.legend()

# 2. Combine angles vs time into one plot
plt.subplot(2, 1, 2)
s = 4  # Point size

if len(rgb_angles) > 0:
    plt.plot(rgb_times, rgb_angles, '-', color='red', lw=lw, label='RGB')
    plt.scatter(rgb_times, rgb_angles, color='red', s=s)

if len(raw_angles) > 0:
    plt.plot(raw_times, raw_angles, '-', color='green', lw=lw, label='RAW')
    plt.scatter(raw_times, raw_angles, color='green', s=s)

if len(dv_angles) > 0:
    plt.plot(dv_times, dv_angles, '-', color='blue', lw=lw, label='DV')
    plt.scatter(dv_times, dv_angles, color='blue', s=s)

plt.title('Barbara Angle vs Time')
plt.xlabel('Timestamp')
plt.ylabel('Angle (deg)')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

## Playback Visualization of Filtered Event Data

In [ ]:
from IPython.display import display, clear_output
import datetime 

def play_filtered_events(events, interval_ms=33, window_size=(230, 230)):
    """
    Play filtered event data
    
    Parameters:
    events: Filtered event data (PyTorch tensor)
    interval_ms: Time interval between frames (milliseconds)
    window_size: Display window size
    """
    # Create event accumulator
    accumulator = dv.Accumulator(window_size)
    
    # Set accumulator parameters
    accumulator.setEventContribution(0.25)
    accumulator.setNeutralPotential(0.5)
    accumulator.setMinPotential(0.0)
    accumulator.setMaxPotential(1.0)
    accumulator.setDecayFunction(dv.Accumulator.Decay.LINEAR)
    accumulator.setDecayParam(1e-6)
    accumulator.setSynchronousDecay(False)
    accumulator.setIgnorePolarity(False)
    
    # Create preview window
    cv2.namedWindow("Events Preview", cv2.WINDOW_NORMAL)
    
    # Create event slicer
    slicer = dv.EventStreamSlicer()
    
    # Frame counter
    frame_counter = 0
    
    def accumulate_events(event_slice):
        nonlocal frame_counter
        
        # Pass event slice to accumulator
        accumulator.accept(event_slice)
        
        # Generate frame
        frame = accumulator.generateFrame()
        
        # Increment frame counter
        frame_counter += 1
        
        # Display frame
        cv2.imshow("Events Preview", frame.image)
        cv2.waitKey(2)
    
    # Set time interval
    slicer.doEveryTimeInterval(datetime.timedelta(milliseconds=interval_ms), accumulate_events)
    
    print("Starting event data playback...")
    
    # Get event data
    # We need to convert PyTorch tensor to event format that dv library can process
    # Process events in batches, 1000 events per batch
    batch_size = 1000
    total_events = len(events)
    
    for i in range(0, total_events, batch_size):
        # Get current batch of events
        batch_events = events[i:min(i+batch_size, total_events)]
        
        # Convert to format that dv library can process
        batch = dv.EventStore()
        for j in range(len(batch_events)):
            event = batch_events[j]
            x = int(event[0].item())
            y = int(event[1].item())
            timestamp = int(event[2].item())
            polarity = int(event[3].item())
            batch.push_back(dv.Event(x, y, timestamp, polarity))
        
        # Pass events to slicer
        slicer.accept(batch)
        
        # Check if 'q' key is pressed to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    print(f"Total frames: {frame_counter}")
    print("Event playback completed")
    cv2.destroyAllWindows()


# Play events
# play_filtered_events(filtered_events_dv, 33, (DV_BOX_SIZE_FOR_EVENT, DV_BOX_SIZE_FOR_EVENT))
play_filtered_events(filtered_events_raw, 33, (RAW_BOX_SIZE_FOR_EVENT, RAW_BOX_SIZE_FOR_EVENT))
# play_filtered_events(filtered_events_rgb, 33, (RGB_BOX_SIZE_FOR_EVENT, RGB_BOX_SIZE_FOR_EVENT))